### Seemingly Unrelated Regressions (SUR)

### Contoh 1

In [6]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.regression.linear_model import OLS
from statsmodels.sandbox.sysreg import SUR
from linearmodels.system import SUR


In [7]:
# Simulasi data
np.random.seed(0)
n = 100
X1 = np.random.rand(n, 2)
X2 = np.random.rand(n, 2)
beta1 = np.array([1, 2])
beta2 = np.array([3, -1])

# Generate data untuk dua persamaan regresi
y1 = X1 @ beta1 + np.random.normal(0, 1, n)
y2 = X2 @ beta2 + np.random.normal(0, 1, n)

# Gabungkan data dalam bentuk DataFrame
data = pd.DataFrame({
    'y1': y1,
    'x1_1': X1[:, 0],
    'x1_2': X1[:, 1],
    'y2': y2,
    'x2_1': X2[:, 0],
    'x2_2': X2[:, 1],
})

# Definisikan persamaan SUR
equations = {
    "y1": "y1 ~ x1_1 + x1_2",
    "y2": "y2 ~ x2_1 + x2_2"
}

# Buat dan fit model SUR
model = SUR.from_formula(equations, data)
result = model.fit()
print(result.summary)

                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.6801
No. Equations.:                     2   McElroy's R-squared:                 0.4151
No. Observations:                 100   Judge's (OLS) R-squared:             0.4213
Date:                Mon, Nov 04 2024   Berndt's R-squared:                  0.6555
Time:                        13:43:26   Dhrymes's R-squared:                 0.6704
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
                     Equation: y1, Dependent Variable: y1                     
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
x1_1           0.5666     0.2648     2.1395     0.0324      0.0476      1.0856
x1_2        

### Contoh 2

In [9]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.sysreg import SUR
from linearmodels.system import SUR

In [13]:
import numpy as np
import pandas as pd
from linearmodels.system import SUR

# Simulasi data
np.random.seed(1)
n = 100

# Variabel independent
GDP = np.random.normal(50000, 10000, n) # GDP dalam ribuan
interest_rate = np.random.normal(5, 1, n) # Interest rate dalam persen

# Parameter yang sebenarnya
alpha1, beta1, gamma1 = 2000, 0.5, -150
alpha2, beta2, gamma2 = 500, 0.3, -100

# Error terms with correlation
error1 = np.random.normal(0, 1000, n)
error2 = error1 * 0.5 + np.random.normal(0, 1000, n)  # Correlated error

# Model equations
consumption = alpha1 + beta1 * GDP + gamma1 * interest_rate + error1
investment = alpha2 + beta2 * GDP + gamma2 * interest_rate + error2

# Creating DataFrame
data = pd.DataFrame({
    'consumption': consumption,
    'investment': investment,
    'GDP': GDP,
    'interest_rate': interest_rate
})

# Define model SUR with formula using a dictionary
equations = {
    'consumption': 'consumption ~ GDP + interest_rate',
    'investment': 'investment ~ GDP + interest_rate'
}

# Create and fit the SUR model
model = SUR.from_formula(equations, data)
result = model.fit()
print(result.summary)

                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.9976
No. Equations.:                     2   McElroy's R-squared:                 0.9072
No. Observations:                 100   Judge's (OLS) R-squared:             0.9168
Date:                Mon, Nov 04 2024   Berndt's R-squared:                  0.9516
Time:                        13:53:35   Dhrymes's R-squared:                 0.9975
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
              Equation: consumption, Dependent Variable: consumption             
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
GDP               0.5059     0.0090     56.482     0.0000      0.4883      0.5234


Analisis dari dua tahap model regresi ini adalah sebagai berikut:

## 1. Hasil Pertama (Regresi OLS Terpisah):
   - Pada tahap awal, model regresi untuk konsumsi dan investasi dilakukan secara terpisah menggunakan metode Ordinary Least Squares (OLS).
   - Hasil dari regresi terpisah ini menunjukkan bahwa estimasi koefisien cenderung memiliki **kesalahan (error) yang lebih besar**, terutama karena tidak mempertimbangkan adanya korelasi antara error pada persamaan konsumsi dan investasi.
   - Karena konsumsi dan investasi seringkali dipengaruhi oleh faktor eksternal yang sama, ada kemungkinan error di antara keduanya berkorelasi. Tanpa mempertimbangkan korelasi ini, model OLS standar tidak mampu memberikan estimasi yang seefisien mungkin.

## 2. Hasil Kedua (Model SUR):
   - Pada tahap kedua, dilakukan estimasi model menggunakan *Seemingly Unrelated Regression* (SUR), yang mengatasi kelemahan dari model regresi terpisah dengan memperhitungkan korelasi antar error.
   - SUR memungkinkan persamaan konsumsi dan investasi untuk diestimasi bersama, sehingga **mengurangi error** secara keseluruhan dan menghasilkan estimasi yang lebih tepat dan efisien.
   - Dalam model SUR ini, karena korelasi antar error sudah diakomodasi, hasil estimasi koefisien menunjukkan **tingkat ketepatan yang lebih tinggi** dibandingkan dengan regresi OLS terpisah.

## Kesimpulan dari Analisis SUR:
   - SUR memberikan hasil yang lebih baik dibandingkan dengan model regresi terpisah, terutama dalam konteks di mana persamaan dalam model memiliki error yang berkorelasi.
   - Dengan memperhitungkan korelasi antar error, model SUR menghasilkan estimasi koefisien yang lebih reliabel dan mengurangi error residu secara keseluruhan.
   - Ini berarti bahwa SUR lebih sesuai digunakan dalam situasi di mana terdapat beberapa persamaan yang secara teori atau empiris memiliki error yang saling berkaitan, seperti konsumsi dan investasi dalam perekonomian.

## Rekomendasi:
   - SUR bisa dipilih sebagai metode yang lebih akurat untuk analisis variabel yang saling terkait, terutama bila ada bukti korelasi antar error.